In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

from mnist_tools import *

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


MNIST pixels have values ranging from [0, 255], so we must rescale the data:
* divide all values by 255 so the range is [0, 1] $\rightarrow$ This is implicitly done with .ToTensor()
* standardize the distribution so the mean = 0, stdev = 1  $\rightarrow$ using the mean and stdev from the re-scaled dataset

In [3]:
transformation = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transformation)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transformation)

train_size = int(0.8 * len(train_set))
val_size = len(train_set) - train_size
train_set, val_set = random_split(train_set, [train_size, val_size])

batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [4]:
model = LeNet()
model.to(device)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [5]:
epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

avg_train_losses = []
avg_val_losses = []

for epoch in range(epochs):
    avg_tloss, avg_vloss = train(model, train_loader, val_loader, criterion, optimizer, device)
    avg_train_losses.append(avg_tloss)
    avg_val_losses.append(avg_vloss)

accuracy = test(model, test_loader, device)
print('Accuracy of model on test set: {:.4f}%'.format(accuracy))

LOSS train 0.767261855661869 valid 0.2173527851899465
LOSS train 0.158371450415502 valid 0.12633562380820512
LOSS train 0.10778249516462286 valid 0.10291131151591738
LOSS train 0.08452287970654046 valid 0.0954701740226398
LOSS train 0.07012435756709116 valid 0.07882866157808652
Accuracy on test set: 0.9804
Accuracy of model on test set: 0.9804%
